<a href="https://colab.research.google.com/github/RNy7846/Mochila_Binaria/blob/main/MochilaBinaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mip

In [ ]:
from mip import *
import random

In [ ]:
# Constantes
RANDOM_SEED   = 2
NUM_PRODUTOS  = 10
NUM_MOCHILAS  = 2

In [ ]:
# Gerar produtos com peso e valor aleatórios
def gerar_produtos(prod):
    random.seed(RANDOM_SEED)
    for i in range(NUM_PRODUTOS):
        cod = 'p_{}'.format(i)
        valor = random.choice(range(1,10)) # Valor em reais
        peso = random.choice(range(100,999)) # Valor em reais
        prod[cod] = {
            'valor': valor,
            'peso': peso
        }

In [ ]:
# Gera mochilas com pesos aleatórias
def gerar_mochilas(moc):
    random.seed(RANDOM_SEED)
    for i in range(NUM_MOCHILAS):
        cod = 'm_{}'.format(i)
        carga_maxima = random.choice(range(500,2000)) # Carga máxima em gramas
        moc[cod] = {
            'carga_maxima': carga_maxima
        }

In [ ]:

# Variável que armazena os Dados
produtos = {}
mochilas = {}
gerar_produtos(produtos)
gerar_mochilas(mochilas)


In [ ]:
# Imprimir os produtos
def imprimir_produtos(prod):
    print("PRODUTOS")
    print("Cód \tValor \t\tPeso")
    for p in prod:
        print("{}\tR$ {},00 \t{}g".format(p, prod[p]['valor'], prod[p]['peso']))

In [ ]:
# Todos os produtos
imprimir_produtos(produtos)

PRODUTOS
Cód 	Valor 		Peso
p_0	R$ 1,00 	193g
p_1	R$ 2,00 	469g
p_2	R$ 3,00 	853g
p_3	R$ 5,00 	357g
p_4	R$ 4,00 	721g
p_5	R$ 1,00 	695g
p_6	R$ 3,00 	541g
p_7	R$ 7,00 	922g
p_8	R$ 9,00 	480g
p_9	R$ 9,00 	555g


In [ ]:
# Imprime as mochilas
def imprimir_mochilas(moc):
    print("\nMOCHILAS")
    print("Cód \tCarga Máxima")
    for m in moc:
        print("{} \t{}g".format(m, moc[m]['carga_maxima']))

In [ ]:
# Todas as mochilas
imprimir_mochilas(mochilas)


MOCHILAS
Cód 	Carga Máxima
m_0 	615g
m_1 	687g


In [ ]:
# Modelo 
modelo = Model(sense=MAXIMIZE)

In [ ]:
# Variáveis de decisão
carga = {} # qual produto será colcoado em cada mochila

In [ ]:
for m in mochilas:
    for p in produtos:
        carga[(m, p)] = modelo.add_var(var_type=BINARY)     
        
#O mesmo produto não pode ser colocado nas duas mochilas
for p in produtos:        
    modelo += xsum(carga[(m, p)] for m in mochilas) <= 1

#A soma dos pesos dos produtos alocados em uma mochila não devem ser maior do que a carga máxima suportada pela mochila
for m in mochilas:    
    modelo += xsum(carga[(m, p)] * produtos[p]['peso'] for p in produtos) <= mochilas[m]['carga_maxima']

# Função Objetivo
modelo.objective = maximize(
    xsum(carga[(m, p)] * produtos[p]['valor']
         for m in mochilas 
             for p in produtos
         if (m, p) in carga
    )
)

modelo.optimize()


<OptimizationStatus.OPTIMAL: 0>

In [ ]:
# Resultado
print("\n=====")
print("Valor Total em Todas as Mochilas {}".format(modelo.objective_values))
print("=====")
for m in mochilas:
    print("\nCarga da Mochila {} com capacidade de {}g".format(m, mochilas[m]['carga_maxima']))
    valor_total = 0
    carga_total = 0
    for p in produtos:
        if (carga[(m, p)].x == 1):
            valor_total += produtos[p]['valor']
            carga_total += produtos[p]['peso']
            print("{} \tR$ {},00 \t{}g".format(p, produtos[p]['valor'], produtos[p]['peso']))
    
    print("-\nValor Total: R$ {},00 \nCarga Total: {}g\nCapacidade Ociosa: {}g".format(valor_total, carga_total, (mochilas[m]['carga_maxima']-carga_total)))


=====
Valor Total em Todas as Mochilas [-19.0]
=====

Carga da Mochila m_0 com capacidade de 615g
p_9 	R$ 9,00 	555g
-
Valor Total: R$ 9,00 
Carga Total: 555g
Capacidade Ociosa: 60g

Carga da Mochila m_1 com capacidade de 687g
p_0 	R$ 1,00 	193g
p_8 	R$ 9,00 	480g
-
Valor Total: R$ 10,00 
Carga Total: 673g
Capacidade Ociosa: 14g
